# Naive Bayesian Network 

## Introduction
Using the Enron email data set, we will create a Naive Bayesian network in this simple exercise to classify whether a given email is a spam or ham by looking at its word frequency as feature set.

## Mathematics
### Definitions

Let's define $N$ to be the number of total emails we have in the dataset and $N_{s}$ to be the number of spam emails in the email set.

$N_{so}$ is the number of spam emails that contain the word "offer"

$N_{o}$ is the number of emails that contain the word "offer"

Then the probability of having a spam email in the set is said to be:

$$
P(SPAM=1) = \frac{N_{s}}{N}
$$ 

And the probability of having an email that contains the word *offer* is:

$$
P(OFFER=1) = \frac{N_{o}}{N}
$$

Finally, the conditional probability of an email being a spam email given that it contains the word *offer*:

$$
P(SPAM=1\mid OFFER=1) := \frac{N_{so}}{N_{o}}
$$

### Postulate

If the probability of finding the word *offer* given that it's a spam email is higher than that of finding the word *offer* in a non-spam email:

$$
P(OFFER =1 \mid SPAM=1)  > P(OFFER = 1 \mid SPAM=0)
$$

then we can infer that:

$$
P(SPAM=1 \mid OFFER=1) > P(SPAM = 1)
$$

### Proof

$$
P(SPAM=1 \mid OFFER=1) = \frac{P(OFFER=1 \mid SPAM=1)P(SPAM=1)}{P(OFFER=1)} = \frac{\frac{N_{so}}{N_{s}}\frac{N_{s}}{N}}{\frac{N_{o}}{N}} = \frac{N_{so}}{N_{o}}
$$


This is known as the **Bayes' rule**, famously stated as $P(A \mid B)=\frac{P(B \mid A)P(A)}{P(B)}$

$$
P(SPAM=0 \mid OFFER=1) = \frac{P(OFFER=1 \mid SPAM=0)P(SPAM=0)}{P(OFFER=1)} \\
P(SPAM=1 \mid OFFER=1) = \frac{P(OFFER=1 \mid SPAM=1)P(SPAM=1)}{P(OFFER=1)}
$$



For abbreviation, let's define that:

$$
P(SPAM=1) := P(S) \\
P(OFFER=1 \mid SPAM=1) := P(O \mid S) \\
P(OFFER=1 \mid SPAM=0) := P(O \mid S_{c}) \\
P(SPAM=1 \mid OFFER=1):= P(S \mid O)
$$

Begin with

$$
P(O \mid S) > P(O \mid S_{c})
$$

Rewrite them using **Bayes' rule**:

$$
\frac{P(S \mid O) P(O)}{P(S)} > \frac{P(S_{c} \mid O)P(O)}{P(S_{c})}
$$

The $P(O)$ terms cancel out each other:

$$
\frac{P(S \mid O)}{P(S)} > \frac{P(S_{c} \mid O)}{P(S_{c})}
$$

By definition, we can rewrite the right hand side as the following:

$$
\frac{P(S \mid O)}{P(S)} > \frac{1 - P(S \mid O)}{1 - P(S)}
$$

Re-organize the terms:

$$
\frac{1 - P(S)}{P(S)} > \frac{1 - P(S \mid O)}{P(S \mid O)}
$$

Then we can easily see that:

$$
\frac{1}{P(S)} - 1 > \frac{1}{P(S \mid O)} - 1 \\
\frac{1}{P(S)} > \frac{1}{P(S \mid O)} \\
$$

**Q.E.D.**
$$
P(S \mid O) > P(S)
$$

## Feature Probability
First of all, we load the data into a class object called `EmailSet` and compute the feature probability for each word that has appeared in the email using `FeatureProbability.from_email_set`.

In [1]:
from naive_bayes.email_set import EmailSet
from naive_bayes.email_set import build_and_save_email_set
from naive_bayes.feature_prob_set import FeatureProbabilitySet

# If you haven't pickled it, then run 
build_and_save_email_set()

es = EmailSet.get()
fps = FeatureProbabilitySet.from_email_set(es)

print "Feature probability set has %s ham emails." % fps.class_count.ham_count
print "Feature probability set has %s spam emails." % fps.class_count.spam_count

Dataset already processed!
Feature probability set has 3672 ham emails.
Feature probability set has 1500 spam emails.


In [2]:
code = es.word_encoding_dictionary.word_to_code("offer")
print "Code:%s with count: %s" % (code, fps.code_count[code])
print "Prob ratio: %s" % fps.code_prob_ratio(code)

Code:3751 with count: {'spam_count': 141, 'ham_count': 61}
Prob ratio: 5.65849180328


### Edge cases

In [3]:
code = es.word_encoding_dictionary.word_to_code("compensating")
print "Code:%s with count: %s" % (code, fps.code_count[code])
print "Prob ratio: %s" % fps.code_prob_ratio(code)

Code:14526 with count: {'spam_count': 0, 'ham_count': 1}
Prob ratio: 0.0


In [4]:
code = es.word_encoding_dictionary.word_to_code("bacterial")
print "Code:%s with count: %s" % (code, fps.code_count[code])
print "Prob ratio: %s" % fps.code_prob_ratio(code)

Code:20347 with count: {'spam_count': 1, 'ham_count': 0}
Prob ratio: inf


Notice that the word *bacterial* and *compensating* have rare occurence in the data set. The probability we compute has a very noisy estimate for their true value. In other words, they are statistically insignificant for us to draw any reliable conclusion. It is not safe to make the assumption that every email with teh word *bacterial* is a spam email.

### Filter Low-reach Features
Let's apply a limit to filter the words that have very low occurence in our data set.

In [8]:
from naive_bayes.feature_prob_selector import FeatureProbabilitySelector
fps = FeatureProbabilitySet.from_email_set(es).filter_low_reach(limit=100)
best_spam_features = FeatureProbabilitySelector.best_spam_features(fps)
best_ham_features = FeatureProbabilitySelector.best_ham_features(fps)

print "Best Spam Features"
FeatureProbabilitySelector.print_feature_list(best_spam_features, es.word_encoding_dictionary)
print "\n"
print "Best Ham Features"
FeatureProbabilitySelector.print_feature_list(best_ham_features, es.word_encoding_dictionary)

Best Spam Features
18629 | 2004 | {'spam_count': 121, 'ham_count': 1} | 296.208
2252 | microsoft | {'spam_count': 98, 'ham_count': 11} | 21.8094545455
5912 | investment | {'spam_count': 96, 'ham_count': 11} | 21.3643636364
2993 | results | {'spam_count': 98, 'ham_count': 18} | 13.328
4144 | v | {'spam_count': 134, 'ham_count': 26} | 12.6166153846
1123 | million | {'spam_count': 97, 'ham_count': 20} | 11.8728
4335 | stop | {'spam_count': 147, 'ham_count': 31} | 11.6082580645
6730 | software | {'spam_count': 101, 'ham_count': 22} | 11.2385454545
2189 | 80 | {'spam_count': 104, 'ham_count': 23} | 11.0692173913
515 | dollars | {'spam_count': 113, 'ham_count': 26} | 10.6393846154
1035 | remove | {'spam_count': 110, 'ham_count': 28} | 9.61714285714
7768 | stock | {'spam_count': 84, 'ham_count': 22} | 9.34690909091
6072 | removed | {'spam_count': 83, 'ham_count': 22} | 9.23563636364
674 | money | {'spam_count': 187, 'ham_count': 50} | 9.15552
1089 | world | {'spam_count': 124, 'ham_count': 34

## Using All Features

### Unconditional Independence
Two variables are unconditionally independent, if knowing the result of one tells nothing of the other under any circumstance.

For example, let `H` to be the event of flipping a head, and `S` to be the event of rolling a 6.

$$
P(S \wedge  H) = P(S)P(H) \\
P(H \mid S) = P(H) \\
P(S \wedge H) = P(S)P(H \mid S) = P(S)P(H)
$$